In [10]:
import pandas as pd

# URL do dataset Wine
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'

# Nome das colunas em português BR
column_names_pt_br = [
    'classe', 'alcool', 'acido_malico', 'cinzas', 'alcalinidade_de_cinzas', 'magnesio',
    'fenois_totais', 'flavanoides', 'fenois_nao_flavanoides', 'proantocianinas',
    'intensidade_de_cor', 'matiz', 'od280_od315_de_vinhos_diluidos', 'prolina'
]

# Ler o arquivo CSV com as colunas especificadas
vinhos = pd.read_csv(url, names=column_names_pt_br, dtype={'classe': object})



Pré-processamento dos Dados

In [11]:
X = vinhos.drop('classe', axis=1)
y = vinhos['classe']

# Normalizar os dados
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


Configuração do Experimento

In [12]:
# Definir valores de random_state para os experimentos
random_states = [42, 17, 24]

# Armazenar os resultados
results = {}


Estrutura de Cross-Validation com KFold

In [13]:
import numpy as np  # Importar numpy para calcular média e desvio padrão
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Loop para testar diferentes random_states
for random_state in random_states:
    # Configuração do KFold com 10 splits
    kf = KFold(n_splits=10, shuffle=True, random_state=random_state)

    # Listas para armazenar as acurácias
    accuracies_k3 = []
    accuracies_k5 = []

    # Loop para cada fold
    for train_index, test_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        knn_3 = KNeighborsClassifier(n_neighbors=3)
        knn_3.fit(X_train, y_train)
        y_pred_3 = knn_3.predict(X_test)
        accuracy_3 = accuracy_score(y_test, y_pred_3)
        accuracies_k3.append(accuracy_3)

        knn_5 = KNeighborsClassifier(n_neighbors=5)
        knn_5.fit(X_train, y_train)
        y_pred_5 = knn_5.predict(X_test)
        accuracy_5 = accuracy_score(y_test, y_pred_5)
        accuracies_k5.append(accuracy_5)

    # Armazenar a média e desvio padrão das acurácias usando numpy
    results[random_state] = {
        'k=3_mean': np.mean(accuracies_k3),
        'k=3_std': np.std(accuracies_k3),
        'k=5_mean': np.mean(accuracies_k5),
        'k=5_std': np.std(accuracies_k5)
    }


Análise dos Resultados e Exibição

In [14]:
# Organizar os resultados em um DataFrame para exibição
df_results = pd.DataFrame(results).T
df_results.index.name = 'random_state'

# Tabela com os resultados
print(df_results)

# Análise
for random_state in random_states:
    k3_mean = df_results.loc[random_state, 'k=3_mean']
    k5_mean = df_results.loc[random_state, 'k=5_mean']

    print(f'')

    if k3_mean > k5_mean:
        print(f'Para random_state {random_state}, o melhor k é 3 com acurácia média de {k3_mean:.4f}')
    elif k5_mean > k3_mean:
        print(f'Para random_state {random_state}, o melhor k é 5 com acurácia média de {k5_mean:.4f}')
    else:
        print(f'Para random_state {random_state}, houve empate nas acurácias médias entre k=3 e k=5.')


              k=3_mean   k=3_std  k=5_mean   k=5_std
random_state                                        
42            0.949346  0.039951  0.971895  0.028120
17            0.954575  0.061875  0.966013  0.045616
24            0.942810  0.063288  0.960131  0.056806

Para random_state 42, o melhor k é 5 com acurácia média de 0.9719

Para random_state 17, o melhor k é 5 com acurácia média de 0.9660

Para random_state 24, o melhor k é 5 com acurácia média de 0.9601
